In [1]:
import pandas as pd
import numpy as np
import json
from math import ceil

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [2]:
f = open("usage_stats_manual_takeout_13_12.json")
usage_results = json.load(f)
usage_results_objects = []
for result in usage_results:
    usage_results_objects.append(result["_source"])

In [3]:
def get_documents_data_into_df(
    results,
    fields,
) -> pd.DataFrame:
    """
    Returns pandas dataframe, that contains one datatype object per row with columns specified in
    the field parameter.
    """

    def get_field_data(input_doc, field):
        for field in field.split("."):
            input_doc = input_doc.get(field)
        return input_doc

    return_df = pd.DataFrame(columns=[field.split(".")[-1] for field in fields])
    for result in results:
        document = result
        temp_df = pd.DataFrame(
            {field.split(".")[-1]: get_field_data(document, field) for field in fields}, index=range(1)
        )
        return_df = pd.concat([return_df, temp_df]).reset_index(drop=True)

    return return_df.fillna(value=np.nan)

In [4]:
fields = ["timestamp", "report_timeframe", "usage_statistics_result.active_users", 
          "usage_statistics_result.new_users_registered", "usage_statistics_result.total_users_registered",
         "usage_statistics_result.unique_users_that_logged_event"]
usage_df = get_documents_data_into_df(usage_results_objects, fields)

/tmp/ipykernel_25940/1673485834.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return return_df.fillna(value=np.nan)


In [5]:
usage_df["day_of_the_week"] = pd.to_datetime(usage_df["timestamp"]).dt.day_name()
usage_df["timestamp"] = pd.to_datetime(usage_df["timestamp"]).dt.date

In [6]:
df_list = []
diary_events_values = []
diary_notes_values = []
diary_ratings_values = []
top3_users_diary_events = []
for result in usage_results_objects:
    timestamp = result["timestamp"]
    document_count_df_dict = result["usage_statistics_result"]["document_count_df"]
    temp_df = pd.DataFrame(document_count_df_dict)
    temp_df = temp_df.set_index("DataType")
    df_list.append(temp_df)
    diary_events_values.append(temp_df.loc["DiaryEvents"]["Document_Count"])
    diary_notes_values.append(temp_df.loc["DiaryNotes"]["Document_Count"])
    diary_ratings_values.append(temp_df.loc["DiaryRating"]["Document_Count"])
    top3_users_diary_events.append(sum(result["usage_statistics_result"]["top10_diary_events_users"][:3]))

In [7]:
usage_df["DiaryEvents"] = pd.Series(diary_events_values)
usage_df["DiaryNotes"] = pd.Series(diary_notes_values)
usage_df["DiaryRating"] = pd.Series(diary_ratings_values)
usage_df["Top3UsersDiaryEvents"] = pd.Series(top3_users_diary_events)
usage_df["PortionOfEventsByTop3"] = usage_df["Top3UsersDiaryEvents"] / usage_df["DiaryEvents"] * 100
usage_df["PortionOfEventsByTop3"] = usage_df["PortionOfEventsByTop3"].round(2)

In [8]:
usage_df = usage_df[['timestamp', 'day_of_the_week', 'report_timeframe', 'active_users', 'new_users_registered',
       'total_users_registered', 'unique_users_that_logged_event',
       'DiaryEvents', 'DiaryNotes', 'DiaryRating',
       'Top3UsersDiaryEvents', 'PortionOfEventsByTop3']]

In [9]:
usage_df["active_without_new"] = usage_df["active_users"] - usage_df["new_users_registered"]

In [10]:
usage_df[usage_df["report_timeframe"] == "D"]

,timestamp,day_of_the_week,report_timeframe,active_users,new_users_registered,total_users_registered,unique_users_that_logged_event,DiaryEvents,DiaryNotes,DiaryRating,Top3UsersDiaryEvents,PortionOfEventsByTop3,active_without_new
0,2024-02-22,Thursday,D,76,17,15664,13,43,20,18,25,58.14,59
1,2024-02-21,Wednesday,D,79,25,15647,14,55,26,28,29,52.73,54
2,2024-02-20,Tuesday,D,95,24,15622,16,62,19,23,30,48.39,71
3,2024-02-19,Monday,D,75,23,15598,10,34,26,25,20,58.82,52
5,2024-02-18,Sunday,D,75,17,15575,14,57,19,17,28,49.12,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2023-09-16,Saturday,D,34,9,3897,18,54,12,16,33,61.11,25
170,2023-09-15,Friday,D,35,10,3888,17,78,15,17,44,56.41,25
171,2023-09-14,Thursday,D,36,6,3878,18,86,8,12,52,60.47,30
172,2023-09-13,Wednesday,D,29,6,3873,15,60,8,12,33,55.00,23


In [11]:
usage_df[usage_df["report_timeframe"] == "D"].to_csv("daily.csv")

In [12]:
usage_df.to_csv("all_reports.csv")

In [13]:
import matplotlib.pyplot as plt

ImportError: cannot import name '_imaging' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)

In [14]:
usage_df.plot(x='timestamp', y=["active_without_new", "total_users_registered"],  kind='line')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [14]:
pip install PIL

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL
Note: you may need to restart the kernel to use updated packages.
